In [1]:
if "running_all" not in globals():
    from ipywidgets import widgets
    matrices = ["ex10", "msc04515", "s1rmq4m1", "Na5", "bcsstk18",
                "vibrobox", "cbuckle", "Pres_Poisson", "raefsky4", "vanbody",
                "ct20stif", "cant", "bcircuit", "apache1", "consph"]
    b = widgets.Button(description="Run over all matrices", button_style="success")
    output = widgets.Output()

    display(b, output)

    def run_over_all_matrices(button):
        global running_all
        global matrix
        running_all = True
        with output:
            for matrix in matrices:
                print(f"Running {matrix}...")
                %run ./protection_vis.ipynb # will output at this cell rather than later
            print("Finished!")
    b.on_click(run_over_all_matrices)

Button(button_style='success', description='Run over all matrices', style=ButtonStyle())

Output()

In [ ]:
if "running_all" not in globals():
    matrix = "cant"  # manually set to run over one matrix
matrix

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from json import load
sys.path.append(os.path.join(os.getcwd(), os.pardir))
from io_utils import load_matrices_from_dir, load_models
from sklearn.metrics import r2_score

In [ ]:
TESTING_DATA_SIZE = 1000

df = pd.read_csv(f"data/{matrix}_{TESTING_DATA_SIZE}.csv")
X_test = df[["error_iter", "pos_2norm"]].to_numpy()
y_test = df[["slowdown"]].to_numpy()
df.head()

In [ ]:
mats = load_matrices_from_dir("../matrices/raw", subset=[matrix])
mat = list(mats.values())[0]
mat

In [ ]:
mat_errorfree_iters = {
    "apache1": 131, 
    "bcircuit": 48, 
    "bcsstk18": 432, 
    "cant": 394,
    "cbuckle": 341, 
    "consph": 611, 
    "ct20stif": 364, 
    "ex10": 479,
    "msc04515": 1849, 
    "Na5": 101, 
    "Pres_Poisson": 72,
    "raefsky4": 79, 
    "s1rmq4m1": 605, 
    "vanbody": 491, 
    "vibrobox": 565,
}
errorfree_iterations = mat_errorfree_iters[matrix]

In [ ]:
chosen_models = {
    "apache1": "XGBRegressor",
    "bcircuit": "XGBRegressor",
    "bcsstk18": "XGBRegressor",
    "cant": "KNeighborsRegressor",
    "cbuckle": "KNeighborsRegressor",
    "consph": "Ridge",
    "ct20stif": "XGBRegressor",
    "ex10": "RandomForestRegressor",
    "msc04515": "RandomForestRegressor",
    "Na5": "XGBRegressor",
    "Pres_Poisson": "XGBRegressor",
    "raefsky4": "RandomForestRegressor",
    "s1rmq4m1": "RandomForestRegressor",
    "vanbody": "RandomForestRegressor",
    "vibrobox": "KNeighborsRegressor",
}

model_path = f"./models/{matrix}/best_{chosen_models[matrix]}.pkl"
model = load_models([model_path])[0]
df["output"] = model.predict(X_test)
model

In [ ]:
with open(f"../matrices/2norms/{matrix}_pos_2norms.json") as f:
    pos_2norms = load(f)

In [ ]:
# df_piv = df_full.pivot_table(index=["2norm", "rowid"], columns="i", values="output")
# df_piv.head()

In [ ]:
r2 = r2_score(y_test, model.predict(X_test))
r2

In [ ]:
norms = sorted(list(df["pos_2norm"].unique()))
iters = range(errorfree_iterations)
iter_norm_pairs_spaced = np.array([[i, n, p] for p, n in enumerate(norms)
                                   for i in iters])
df_spaced = pd.DataFrame(iter_norm_pairs_spaced,
                         columns=["i", "2norm", "rowid"])
df_spaced["output"] = model.predict(df_spaced[["i", "2norm"]])
df_spaced_piv = df_spaced.pivot(index=["2norm"], columns="i", values="output")

df_spaced_piv.head()

In [ ]:
plt.gcf().set_size_inches(8, 8)
plt.gcf().set_dpi(100)

y, x = np.meshgrid(norms, iters)
norm = plt.Normalize(1, 100)
ax = plt.pcolormesh(x, y, df_spaced_piv.to_numpy().T,
                    cmap="Spectral_r", norm=norm, shading="nearest")
sm = plt.cm.ScalarMappable(cmap="Spectral_r", norm=norm)
sm.set_array([])
ax.figure.colorbar(sm, ax=plt.gca(), label="Predicted Slowdown (x times)")

plt.ylim(min(norms), max(norms))
plt.yscale("log")
plt.gca().invert_yaxis()
plt.xlabel("Error Iteration")
plt.ylabel("Row 2-norm of A")
plt.title(f"{matrix}", weight="bold")
plt.tight_layout()
plt.savefig(f"./figures/{matrix}/iter_r2n_output.png")
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2)
fig.set_size_inches(8, 12)
fig.set_dpi(100)

scatter = sns.scatterplot(df, ax=ax1, x="error_iter", y="pos_2norm",
                     hue="slowdown", legend=False, palette="Spectral_r")
norm = plt.Normalize(1, 100)
sm = plt.cm.ScalarMappable(cmap="Spectral_r", norm=norm)
sm.set_array([])
scatter.figure.colorbar(sm, ax=scatter, label="Slowdown (x times)")

ax1.set_yscale("log")
ax1.set_ylim(df["pos_2norm"].min(), df["pos_2norm"].max())
scatter.invert_yaxis()
ax1.set_xlabel("Error Iteration")
ax1.set_ylabel("Row 2-norm of A")

y, x = np.meshgrid(norms, iters)
norm = plt.Normalize(1, 100)
heatmap = plt.pcolormesh(x, y, df_spaced_piv.to_numpy().T,
                    cmap="Spectral_r", norm=norm, shading="nearest")
sm = plt.cm.ScalarMappable(cmap="Spectral_r", norm=norm)
sm.set_array([])
heatmap.figure.colorbar(sm, ax=ax2, label="Predicted Slowdown (x times)")

ax2.set_ylim(min(norms), max(norms))
ax2.set_yscale("log")
ax2.invert_yaxis()
ax2.set_xlabel("Error Iteration")
ax2.set_ylabel("Row 2-norm of A")

ax1.set_title(f"{matrix}\n$r^2 = {round(r2, 3)}$", weight="bold")
plt.tight_layout()
plt.savefig(f"./figures/{matrix}/fit_combined.png")
plt.show()

In [ ]:
df_full = pd.DataFrame([[i, pos_2norms[str(pos)], pos] for pos in range(mat.shape[0])
                   for i in range(errorfree_iterations)], columns=["i", "2norm", "rowid"])
df_full["output"] = model.predict(df_full[["i", "2norm"]])


df_piv_rowid = df_full.pivot(index=["rowid"], columns="i", values="output")
df_piv_rowid.head()

In [ ]:
plt.gcf().set_size_inches(8, 8)
plt.gcf().set_dpi(100)

xskip = 20 if errorfree_iterations < 200 else 100
yskip = 1000 if mat.shape[0] < 10000 else 2000 if mat.shape[0] < 20000 else 10000
ax = sns.heatmap(df_piv_rowid, vmin=1, vmax=df_full["output"].max(), cmap="flare_r", xticklabels=xskip,
            yticklabels=yskip, cbar_kws={"label": "Predicted Slowdown (x times)"})

plt.xlabel("Iteration")
plt.ylabel("Row Index")
plt.title(f"{matrix}", weight="bold")
plt.tight_layout()
plt.savefig(f"./figures/{matrix}/protection_scheme.png")
plt.show()